In [76]:
from Signal_generator.generate_signal import generate_X_matrix
from Algorithmes.beamforming import beamforming_method
from Signal_generator.generate_signal import generate_A_matrix
from Signal_generator.generate_signal import generate_noise
from Signal_generator.generate_signal import generate_R_hat
from Signal_generator.generate_signal import generate_R_hat_with_phase
from Signal_generator.generate_signal import generate_R_hat_with_phase_complex
from Algorithmes.music import music_method
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split 
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchsummary import summary
import math
import itertools
# Définition des paramètres du problème

nbSources = 2 # Nombre de sources  dans l'article ils fixent le nombre de sources à 2
nbSensors = 9 # Nombre de capteurs
nbTimePoints = 100 # Nombre de points temporels
var_ratio_1=1 # Rapport d intensité entre les deux signaux



#Pour l'échantillon d'entraînements ils utilisent une observation de chaque combinaison d'angles d'arrivée possible pour chaque SNR level, dans le cas d'une seule source, 2G+1 observations
#SNR ratio dans le papier est seulement low, de -20 à 0 DB avec un pas de 5
#Petite note : si on fait un modèle pour chaque SNR level, les performances augmentent légérements, mais cela ne vaut pas l'effort effectué
signal_noise_ratio = [-20,-15,-10,-5,0] # Rapport signal sur bruit en décibels. Si 'False', cela revient à une absence totale de bruit.
L = nbSensors
T = 100
correlation_List = [0] # Liste des corrélations. Il y a une corrélation nécéssaire pour chaque paire distincte de sources différentes: 0 pour 1 source, 1 pour 2 sources, 3 pour 3 sources, 6 pour 4 sources etc...
# Ordre de remplisage de la correlation_List: de gauche à droite et ligne par ligne, moitié haut-droite de la matrice uniquement, puis symétrie de ces valeurs pour la moitié bas-gauche.


In [77]:


def angles_to_binary_vector(thetaList, G):
    """
    Convertit une liste d'angles en un vecteur binaire représentant la présence/absence d'angles dans la grille G.
    
    Args:
    - thetaList: Liste des angles d'arrivée pour un échantillon.
    - G: Grille des angles possibles.
    
    Returns:
    - Vecteur binaire des labels Z.
    """
    Z = np.zeros(len(G), dtype=int)
    for theta in thetaList:
        index = np.searchsorted(G, theta)  # Trouver l'indice de l'angle dans la grille
        if 0 <= index < len(G):  # S'assurer que l'indice est dans la plage valide
            Z[index] = 1
    return Z


def generate_combinations(phi_max, rho, nb_sources):
    """Cette fonction sert à générer toutes les combinaisons d'angles possibles dans la plage de résolution"""
    # Générer une plage de valeurs possibles pour les signaux
    values = list(range(-int(phi_max / rho), int(phi_max / rho) + 1))
    
    # Générer toutes les combinaisons possibles de signaux
    all_combinations = list(itertools.product(values, repeat=nb_sources))
    
    # Supprimer les combinaisons où l'ordre ne compte pas
    unique_combinations = set()
    for combination in all_combinations:
        sorted_combination = tuple(sorted(combination))
        unique_combinations.add(sorted_combination)
    unique_combinations=list(unique_combinations)
        
    #supprimer les combinaisons ou deux signaux ont la même valeur
    combination_a_supp=[]
    for combination in unique_combinations:
        presents=[]
        for element in combination:
            if element not in presents:
                presents.append(element)
            else:
                combination_a_supp.append(combination)
    for combination in combination_a_supp:
        unique_combinations.remove(combination)
                
        
    
    return unique_combinations

In [78]:

#Paramètres pour la simulation

num_samples = 1000  # Nombre d'échantillons à générer #16290
L = nbSensors  # Nombre de capteurs
T = 100  # Nombre de points temporels
phi_max = 45  # φmax
rho = 1 # Résolution

# Grille des angles
G = np.arange(-phi_max, phi_max + rho, rho)



In [79]:
#Génération du set de donnée qui comprend toutes les combinaisons possibles de tous les angles de notre grille,
# avec tout les ratio signaux sur bruit, toutes les corrélations et toutes les calibrations

def generation_donnees( phi_max : int, rho : float, SNR : list, correlation : list, calibration ):
    angles_combination=generate_combinations(phi_max,rho,nbSources)
    num_samples=len(angles_combination)*len(SNR)
    
    # Initialisation des tableaux pour les données
    all_R_hat_with_phase = np.zeros((num_samples, 3, L, L))
    all_Z = np.zeros((num_samples, len(G)))
    
    #On parcourt les différents SNR possibles
    i=0
    for snr in SNR:
        # Générer les angles d'arrivée et la matrice X comme décrit précédemment
        for angles in angles_combination:
            thetaList=list(angles)
            var_ratio = [var_ratio_1]
            X = generate_X_matrix(nbSources, L, T, thetaList, var_ratio, correlation_List, snr, perturbation_parameter_sd=0)# fonction pour générer X, ajoutez les paramètres nécessaires
            
            # Calculer R_hat étendue avec phase
            R_hat_with_phase = generate_R_hat_with_phase(X)
            all_R_hat_with_phase[i] = R_hat_with_phase
            
            # Calculer les labels Z pour les angles générés
            all_Z[i] = angles_to_binary_vector(thetaList, G)
            i+=1
    return all_R_hat_with_phase, all_Z


all_R_hat_with_phase, all_Z=generation_donnees(phi_max, rho, signal_noise_ratio,correlation_List,[])


            
        
    
    
    

In [80]:
"""
#Génération des données

# Initialisation des tableaux pour les données
all_R_hat_with_phase = np.zeros((num_samples, 3, L, L))
all_Z = np.zeros((num_samples, len(G)))

for i in range(num_samples):
    # Générer les angles d'arrivée et la matrice X comme décrit précédemment
    thetaList = [np.random.uniform(-phi_max, phi_max) for _ in range(nbSources)]  # Exemple de génération d'angles
    var_ratio = [var_ratio_1]
    X = generate_X_matrix(nbSources, L, T, thetaList, var_ratio, correlation_List, 0, perturbation_parameter_sd=0)  # Votre fonction pour générer X, ajoutez les paramètres nécessaires
    
    # Calculer R_hat étendue avec phase
    R_hat_with_phase = generate_R_hat_with_phase(X)
    all_R_hat_with_phase[i] = R_hat_with_phase
    
    # Calculer les labels Z pour les angles générés
    all_Z[i] = angles_to_binary_vector(thetaList, G)
"""

In [81]:
# Division des données en ensembles d'entraînement et de test
X_train, X_test, Z_train, Z_test = train_test_split(all_R_hat_with_phase, all_Z, test_size=0.1 , random_state=42)#test_size=0.1 selon papier

#Conversion des données en tenseur
X_train = torch.from_numpy(X_train).to(dtype=torch.float32)
X_test = torch.from_numpy(X_test).to(dtype=torch.float32)
Z_train = torch.from_numpy(Z_train).to(dtype=torch.float32)
Z_test = torch.from_numpy(Z_test).to(dtype=torch.float32)

In [82]:

#Création du dataset

class DOADataset(Dataset):
    def __init__(self, covariance_matrices, doa_labels):
        """
        Initialisation du Dataset DOA.

        Args:
        - covariance_matrices (np.ndarray): Matrices de covariance étendues avec la phase.
          Forme attendue: (nb_samples, nbSensors, nbSensors, 3) pour les parties réelle, imaginaire, et la phase.
        - doa_labels (np.ndarray): Labels DOA sous forme de vecteurs binaires.
          Forme attendue: (nb_samples, nb_labels), où nb_labels est le nombre de points dans la grille DOA.
        """
        self.covariance_matrices = covariance_matrices
        self.doa_labels = doa_labels

    def __len__(self):
        return len(self.covariance_matrices)

    def __getitem__(self, idx):
      cov_matrix = self.covariance_matrices[idx]
      angle = self.doa_labels[idx]

      # Vérification si cov_matrix est déjà un ndarray, sinon le convertir
      if not isinstance(cov_matrix, np.ndarray):
          cov_matrix = cov_matrix.numpy()  # Convertir en tableau NumPy si ce n'est pas déjà fait
    
      cov_matrix = torch.from_numpy(cov_matrix).float()
      angle = torch.tensor(angle).float()

      return cov_matrix, angle


# Exemple d'utilisation (remplacez R_hat_train/test et angles_train/test par vos données réelles)
train_dataset = DOADataset(X_train, Z_train)
test_dataset = DOADataset(X_test, Z_test)

# Création des DataLoaders pour l'entraînement et le test, batch_size en fonction de l'article
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)



In [83]:
#Définition du device

device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)

print(f"Using {device} device")

Using cpu device


In [84]:

#Définition du modèle
nombre_de_classe=phi_max*2/rho+1
nombre_de_classe=int(nombre_de_classe)
class DOACNN(nn.Module):
    def __init__(self, num_channels=3, num_classes=nombre_de_classe):
        super(DOACNN, self).__init__()

        # Couches convolutionnelles 2D
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=256, kernel_size=3, stride=2)
        self.conv4 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=2, stride=1)
        self.conv7 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=2, stride=1)
        self.conv10 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=2, stride=1)

        # Normalisation de taille 256
        self.norm2 = nn.BatchNorm2d(256)
        self.norm5 = nn.BatchNorm2d(256)
        self.norm8 = nn.BatchNorm2d(256)
        self.norm11 = nn.BatchNorm2d(256)

        self.relu3 =  nn.ReLU()
        self.relu6 =  nn.ReLU()
        self.relu9 =  nn.ReLU()
        self.relu12 =  nn.ReLU()
        self.relu15 =  nn.ReLU()
        self.relu18 =  nn.ReLU()
        self.relu21 =  nn.ReLU()

        # Couches Dropout
        self.dropout16 = nn.Dropout(0.2)
        self.dropout19 = nn.Dropout(0.2)
        self.dropout22 = nn.Dropout(0.2)

        # Couche d'aplatissement
        self.flatten13 = nn.Flatten()

        # Couches entièrement connectées (FC)
        self.fc14 = nn.Linear(in_features=256 , out_features=4096)
        self.fc17 = nn.Linear(in_features=4096, out_features=2048)
        self.fc20 = nn.Linear(in_features=2048, out_features=1024)
        self.fc23 = nn.Linear(in_features=1024, out_features=num_classes)

        # Couche de sortie Sigmoid
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = self.norm2(x)
        x = self.relu3(x)
        x = self.conv4(x)
        x = self.norm5(x)
        x = self.relu6(x)
        x = self.conv7(x)
        x = self.norm8(x)
        x = self.relu9(x)
        x = self.conv10(x)
        x = self.norm11(x)
        x = self.relu12(x)
        x = self.flatten13(x)
        x = self.fc14(x)
        x = self.relu15(x)
        x = self.dropout16(x)
        x = self.fc17(x)
        x = self.relu18(x)
        x = self.dropout19(x)
        x = self.fc20(x)
        x = self.relu21(x)
        x = self.dropout22(x)
        x = self.fc23(x)
        x = self.sigmoid(x)

        return x

# Création d'une instance du modèle
model = DOACNN()
print(model)

DOACNN(
  (conv1): Conv2d(3, 256, kernel_size=(3, 3), stride=(2, 2))
  (conv4): Conv2d(256, 256, kernel_size=(2, 2), stride=(1, 1))
  (conv7): Conv2d(256, 256, kernel_size=(2, 2), stride=(1, 1))
  (conv10): Conv2d(256, 256, kernel_size=(2, 2), stride=(1, 1))
  (norm2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (relu6): ReLU()
  (relu9): ReLU()
  (relu12): ReLU()
  (relu15): ReLU()
  (relu18): ReLU()
  (relu21): ReLU()
  (dropout16): Dropout(p=0.2, inplace=False)
  (dropout19): Dropout(p=0.2, inplace=False)
  (dropout22): Dropout(p=0.2, inplace=False)
  (flatten13): Flatten(start_dim=1, end_dim=-1)
  (fc14): Linear(in_features=256, out_features=4096, bias=T

In [85]:


#Définition des hyperparamètres, choisis en fonction de l'article DOA estimator

learning_rate = 0.001
num_epochs = 200 # à monter à 200 pour le training final selon l'article
loss_fn = nn.BCELoss()  
optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999))
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)

In [101]:
#Boucle d'entraînement

def train_loop(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)

    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Display loss from time to time
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss: >7f}  [{current: >5d} / {size: >5d}]")


#Boucle de tests


def test_loop(dataloader, model, loss_fn, device):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct, MSE = 0, 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            
            #print( f"pred{pred.shape}")
            _, angles_predit = pred.topk(nbSources, dim=1)
            _,angles_correct=top_values, top_indices = y.topk(nbSources, dim=1)
            correct += (angles_predit==angles_correct).type(torch.float).sum().item()
            #Calcul de la somme des carrés des différences entre prédictions et réalité pour la RMSE
            #print( f"MSE initial{MSE}")
            #print( f"angles_predit{angles_predit}")
            taille_du_batch=pred.shape[0]
            for element in range(taille_du_batch):
                for angle in range(nbSources):
                    MSE+=(angles_correct[element, angle]-angles_predit[element, angle])**2

    test_loss /= num_batches
    correct /= size
    MSE=MSE/(len(test_dataloader)*nbSources)
    RMSE=math.sqrt(MSE)
    print(f"\nTest set: \n  Accuracy: {(100 * correct): >0.1f}%, Avg loss: {test_loss: >8f} \n RMSE : {MSE: >8f}")
    return test_loss, correct




In [102]:
#Fonction d'entraînement et tests

def train(model, train_dataloader, test_dataloader, loss_fn, optimizer, epochs, device):
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train_loop(train_dataloader, model, loss_fn, optimizer, device)
        test_loop(test_dataloader, model, loss_fn, device)
        scheduler.step()




In [103]:
train(model, train_dataloader, test_dataloader, loss_fn, optimizer, num_epochs, device)
print('Entraînement terminé!')

Epoch 1
-------------------------------


C:\Users\legra\AppData\Local\Temp\ipykernel_21088\1026075063.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  angle = torch.tensor(angle).float()


loss: 0.028235  [   32 /   900]

Test set: 
  Accuracy: 26.0%, Avg loss: 0.112377 
 RMSE : 17779.750000
Epoch 2
-------------------------------
loss: 0.025215  [   32 /   900]

Test set: 
  Accuracy: 34.0%, Avg loss: 0.115036 
 RMSE : 14707.500000
Epoch 3
-------------------------------
loss: 0.021329  [   32 /   900]

Test set: 
  Accuracy: 32.0%, Avg loss: 0.112623 
 RMSE : 14557.750000
Epoch 4
-------------------------------
loss: 0.025743  [   32 /   900]

Test set: 
  Accuracy: 30.0%, Avg loss: 0.112413 
 RMSE : 13098.125000
Epoch 5
-------------------------------
loss: 0.018212  [   32 /   900]

Test set: 
  Accuracy: 36.0%, Avg loss: 0.112688 
 RMSE : 15084.500000
Epoch 6
-------------------------------
loss: 0.025506  [   32 /   900]

Test set: 
  Accuracy: 33.0%, Avg loss: 0.111970 
 RMSE : 13086.875000
Epoch 7
-------------------------------
loss: 0.021971  [   32 /   900]

Test set: 
  Accuracy: 33.0%, Avg loss: 0.113155 
 RMSE : 16418.625000
Epoch 8
------------------------

KeyboardInterrupt: 

In [ ]:
#Quelques informations sur notre modèle : 

print(model)

summary(model, input_size=(3, nbSensors, nbSensors))

#Poids des paramètres :


# Initialiser le compteur de paramètres
total_params = 0

# Boucle à travers les paramètres de votre modèle
for param in model.parameters():
    # Vérifier si le paramètre nécessite un gradient (c'est-à-dire s'il est entraînable)
    if param.requires_grad:
        # Compter le nombre de valeurs dans le paramètre
        total_params += param.numel()

# Boucle à travers les paramètres de votre modèle
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()}")

DOACNN(
  (conv1): Conv2d(3, 256, kernel_size=(3, 3), stride=(2, 2))
  (conv4): Conv2d(256, 256, kernel_size=(2, 2), stride=(1, 1))
  (conv7): Conv2d(256, 256, kernel_size=(2, 2), stride=(1, 1))
  (conv10): Conv2d(256, 256, kernel_size=(2, 2), stride=(1, 1))
  (norm2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (relu6): ReLU()
  (relu9): ReLU()
  (relu12): ReLU()
  (relu15): ReLU()
  (relu18): ReLU()
  (relu21): ReLU()
  (dropout16): Dropout(p=0.2, inplace=False)
  (dropout19): Dropout(p=0.2, inplace=False)
  (dropout22): Dropout(p=0.2, inplace=False)
  (flatten13): Flatten(start_dim=1, end_dim=-1)
  (fc14): Linear(in_features=256, out_features=4096, bias=T

In [ ]:
#Sauvegarder le modèle pour l'utiliser dans un autre module python:
torch.save(model.state_dict(), 'C:\\Users\\legra\\Documents pc\\ENSAI\\3A\\PFE\\model.pth')


"""
Puis voici la marche à suivre pour charger le modèle entrainé dans un nouveau module python

import torch
import torch.nn as nn

# Définir la classe de votre modèle (avec la même architecture que celle utilisée pour l'entraînement)
class VotreModele(nn.Module):
    def __init__(self, ...):  # Définir les mêmes couches que celles utilisées pour l'entraînement
        super(VotreModele, self).__init__()
        ...

# Créer une instance de votre modèle
model = VotreModele(...)

# Charger les poids du modèle entraîné
model.load_state_dict(torch.load('chemin/vers/votre/modele.pth'))"""

"\nPuis voici la marche à suivre pour charger le modèle entrainé dans un nouveau module python\n\nimport torch\nimport torch.nn as nn\n\n# Définir la classe de votre modèle (avec la même architecture que celle utilisée pour l'entraînement)\nclass VotreModele(nn.Module):\n    def __init__(self, ...):  # Définir les mêmes couches que celles utilisées pour l'entraînement\n        super(VotreModele, self).__init__()\n        ...\n\n# Créer une instance de votre modèle\nmodel = VotreModele(...)\n\n# Charger les poids du modèle entraîné\nmodel.load_state_dict(torch.load('chemin/vers/votre/modele.pth'))"

# Test du modèle complexe

In [ ]:
#Génération des données

# Initialisation des tableaux pour les données
all_R_hat_with_phase_complex = np.zeros((num_samples, 2, L, L))
all_Z_complex = np.zeros((num_samples, len(G)))

for i in range(num_samples):
    # Générer les angles d'arrivée et la matrice X comme décrit précédemment
    thetaList_complex = [np.random.uniform(-phi_max, phi_max) for _ in range(nbSources)]  # Exemple de génération d'angles
    varList_complex = [np.random.uniform(0, 1000000000000) for _ in range(nbSources)]
    X_complex = generate_X_matrix(nbSources, L, T, thetaList_complex, varList_complex, correlation_List, signal_noise_ratio)  # Votre fonction pour générer X, ajoutez les paramètres nécessaires
    
    # Calculer R_hat étendue avec phase
    R_hat_with_phase_complex = generate_R_hat_with_phase_complex(X_complex)
    all_R_hat_with_phase_complex[i] = R_hat_with_phase_complex
    
    # Calculer les labels Z pour les angles générés
    all_Z_complex[i] = angles_to_binary_vector(thetaList_complex, G)

[[6.63159732e+11]]
[[4.82987448e+11]]
[[1.67230587e+11]]
[[3.19478484e+11]]
[[9.74600785e+11]]
[[3.82785229e+10]]
[[7.99166865e+11]]
[[1.50108477e+11]]
[[1.58339496e+11]]
[[4.30877578e+10]]
[[5.93102752e+11]]
[[4.56835864e+11]]
[[6.64769573e+11]]
[[3.71191579e+11]]
[[4.12925805e+11]]
[[6.30419554e+11]]
[[7.09760851e+11]]
[[1.11243105e+11]]
[[2.97458196e+11]]
[[9.73690689e+11]]
[[7.8889563e+11]]
[[8.95380264e+11]]
[[6.43780185e+11]]
[[7.4314307e+11]]
[[3.77719973e+11]]
[[7.74742734e+11]]
[[3.00329296e+11]]
[[8.97970865e+11]]
[[2.48066757e+10]]
[[3.02099447e+11]]
[[2.79137084e+11]]
[[9.68363226e+11]]
[[3.03763758e+10]]
[[6.48619996e+11]]
[[1.50234373e+11]]
[[8.45255006e+10]]
[[6.43650609e+11]]
[[9.01890732e+10]]
[[7.841603e+11]]
[[4.54706853e+11]]
[[7.53903969e+11]]
[[2.57788275e+11]]
[[4.6995895e+11]]
[[4.84678618e+11]]
[[4.73820299e+11]]
[[5.89877483e+11]]
[[3.20940418e+11]]
[[2.43926637e+11]]
[[1.01176451e+10]]
[[6.84743916e+11]]
[[1.58985186e+11]]
[[2.36427574e+11]]
[[9.65711675e+11]

C:\Users\legra\AppData\Local\Temp\ipykernel_25528\2233590894.py:15: ComplexWarning: Casting complex values to real discards the imaginary part
  all_R_hat_with_phase_complex[i] = R_hat_with_phase_complex


[[4.06882189e+11]]
[[6.17359942e+11]]
[[6.02292975e+11]]
[[7.84722122e+11]]
[[5.77885756e+11]]
[[3.3129245e+11]]
[[1.83134671e+11]]
[[5.96001252e+10]]
[[7.45314371e+11]]
[[3.02588074e+11]]
[[4.00734176e+10]]
[[2.59747991e+11]]
[[5.73970082e+11]]
[[8.41225473e+11]]
[[9.55571918e+11]]
[[8.5715119e+10]]
[[4.63253281e+11]]
[[3.39329667e+11]]
[[3.17223883e+11]]
[[3.24261366e+11]]
[[7.98547375e+11]]
[[8.16790022e+11]]
[[3.26666472e+11]]
[[6.33811868e+11]]
[[2.43930298e+11]]
[[6.34906094e+11]]
[[1.45948221e+10]]
[[7.07359304e+11]]
[[9.81885007e+11]]
[[1.78308994e+11]]
[[4.05059453e+10]]
[[3.25968689e+11]]
[[5.74215033e+11]]
[[5.17896082e+11]]
[[1.78358424e+11]]
[[8.56209372e+11]]
[[5.27873433e+11]]
[[6.32391304e+10]]
[[2.76801507e+11]]
[[9.71874422e+11]]
[[6.50417341e+11]]
[[3.54440503e+11]]
[[6.26901896e+11]]
[[4.9910087e+10]]
[[9.62816894e+09]]
[[6.88493476e+11]]
[[1.840195e+10]]
[[4.90175967e+11]]
[[6.56781644e+11]]
[[8.61733108e+11]]
[[7.22717763e+11]]
[[3.79571138e+11]]
[[2.51309712e+11]

In [ ]:
# Division des données en ensembles d'entraînement et de test
X_train_complex, X_test_complex, Z_train_complex, Z_test_complex = train_test_split(all_R_hat_with_phase_complex, all_Z_complex, test_size=0.1 , random_state=42)#test_size=0.1 selon papier

#Conversion des données en tenseur
X_train_complex = torch.from_numpy(X_train_complex).to(dtype=torch.complex64)
X_test_complex = torch.from_numpy(X_test_complex).to(dtype=torch.complex64)
Z_train_complex = torch.from_numpy(Z_train_complex).to(dtype=torch.complex64)
Z_test_complex = torch.from_numpy(Z_test_complex).to(dtype=torch.complex64)

In [ ]:
#Création du dataset

class DOADatasetComplex(Dataset):
    def __init__(self, covariance_matrices, doa_labels):
        """
        Initialisation du Dataset DOA.

        Args:
        - covariance_matrices (np.ndarray): Matrices de covariance étendues avec la phase.
          Forme attendue: (nb_samples, nbSensors, nbSensors, 3) pour les parties réelle, imaginaire, et la phase.
        - doa_labels (np.ndarray): Labels DOA sous forme de vecteurs binaires.
          Forme attendue: (nb_samples, nb_labels), où nb_labels est le nombre de points dans la grille DOA.
        """
        self.covariance_matrices = covariance_matrices
        self.doa_labels = doa_labels

    def __len__(self):
        return len(self.covariance_matrices)

    def __getitem__(self, idx):
      cov_matrix = self.covariance_matrices[idx]
      angle = self.doa_labels[idx]

      # Vérification si cov_matrix est déjà un ndarray, sinon le convertir
      if not isinstance(cov_matrix, np.ndarray):
          cov_matrix = cov_matrix.numpy()  # Convertir en tableau NumPy si ce n'est pas déjà fait
    
      cov_matrix = torch.from_numpy(cov_matrix).to(dtype=torch.complex64)
      angle = torch.tensor(angle).to(dtype=torch.complex64)

      return cov_matrix, angle


# Exemple d'utilisation (remplacez R_hat_train/test et angles_train/test par vos données réelles)
train_dataset_complex = DOADatasetComplex(X_train_complex, Z_train_complex)
test_dataset_complex = DOADatasetComplex(X_test_complex, Z_test_complex)

# Création des DataLoaders pour l'entraînement et le test, batch_size en fonction de l'article
train_dataloader_complex = DataLoader(train_dataset_complex, batch_size=32, shuffle=True)
test_dataloader_complex = DataLoader(test_dataset_complex, batch_size=32, shuffle=False)

In [ ]:
#Définition du modèle

class DOACNNCOMPLEX(nn.Module):
    def __init__(self, num_channels=2, num_classes=phi_max*2+1):
        super(DOACNNCOMPLEX, self).__init__()

        # Couches convolutionnelles 2D
        self.conv1 = nn.Conv2d(in_channels=num_channels, out_channels=256, kernel_size=3, stride=2, bias=True)
        self.conv4 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=2, stride=1, bias=True)
        self.conv7 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=2, stride=1, bias=True)
        self.conv10 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=2, stride=1, bias=True)

        # Normalisation de taille 256
        self.norm2 = nn.BatchNorm2d(256)
        self.norm5 = nn.BatchNorm2d(256)
        self.norm8 = nn.BatchNorm2d(256)
        self.norm11 = nn.BatchNorm2d(256)

        self.relu3 =  nn.ReLU()
        self.relu6 =  nn.ReLU()
        self.relu9 =  nn.ReLU()
        self.relu12 =  nn.ReLU()
        self.relu15 =  nn.ReLU()
        self.relu18 =  nn.ReLU()
        self.relu21 =  nn.ReLU()

        # Couches Dropout
        self.dropout16 = nn.Dropout(0.2)
        self.dropout19 = nn.Dropout(0.2)
        self.dropout22 = nn.Dropout(0.2)

        # Couche d'aplatissement
        self.flatten13 = nn.Flatten()

        # Couches entièrement connectées (FC)
        self.fc14 = nn.Linear(in_features=256 , out_features=4096)
        self.fc17 = nn.Linear(in_features=4096, out_features=2048)
        self.fc20 = nn.Linear(in_features=2048, out_features=1024)
        self.fc23 = nn.Linear(in_features=1024, out_features=num_classes)

        # Couche de sortie Sigmoid
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = self.norm2(x)
        x = self.relu3(x)
        x = self.conv4(x)
        x = self.norm5(x)
        x = self.relu6(x)
        x = self.conv7(x)
        x = self.norm8(x)
        x = self.relu9(x)
        x = self.conv10(x)
        x = self.norm11(x)
        x = self.relu12(x)
        x = self.flatten13(x)
        x = self.fc14(x)
        x = self.relu15(x)
        x = self.dropout16(x)
        x = self.fc17(x)
        x = self.relu18(x)
        x = self.dropout19(x)
        x = self.fc20(x)
        x = self.relu21(x)
        x = self.dropout22(x)
        x = self.fc23(x)
        x = self.sigmoid(x)

        return x

# Création d'une instance du modèle
modelComplex = DOACNNCOMPLEX()
print(modelComplex)

DOACNNCOMPLEX(
  (conv1): Conv2d(2, 256, kernel_size=(3, 3), stride=(2, 2))
  (conv4): Conv2d(256, 256, kernel_size=(2, 2), stride=(1, 1))
  (conv7): Conv2d(256, 256, kernel_size=(2, 2), stride=(1, 1))
  (conv10): Conv2d(256, 256, kernel_size=(2, 2), stride=(1, 1))
  (norm2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (relu6): ReLU()
  (relu9): ReLU()
  (relu12): ReLU()
  (relu15): ReLU()
  (relu18): ReLU()
  (relu21): ReLU()
  (dropout16): Dropout(p=0.2, inplace=False)
  (dropout19): Dropout(p=0.2, inplace=False)
  (dropout22): Dropout(p=0.2, inplace=False)
  (flatten13): Flatten(start_dim=1, end_dim=-1)
  (fc14): Linear(in_features=256, out_features=4096,

In [ ]:
train(modelComplex, train_dataloader_complex, test_dataloader_complex, loss_fn, optimizer, num_epochs, device)
print('Entraînement terminé!')

Epoch 1
-------------------------------


C:\Users\legra\AppData\Local\Temp\ipykernel_25528\3472206976.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  angle = torch.tensor(angle).to(dtype=torch.complex64)


RuntimeError: Input type (struct c10::complex<float>) and bias type (float) should be the same

In [ ]:
#Quelques informations sur notre modèle : 

print(modelComplex)

summary(modelComplex, input_size=(2, nbSensors, nbSensors))

#Poids des paramètres :


# Initialiser le compteur de paramètres
total_params_complex = 0

# Boucle à travers les paramètres de votre modèle
for param in modelComplex.parameters():
    # Vérifier si le paramètre nécessite un gradient (c'est-à-dire s'il est entraînable)
    if param.requires_grad:
        # Compter le nombre de valeurs dans le paramètre
        total_params_complex += param.numel()

# Boucle à travers les paramètres de votre modèle
for name, param in modelComplex.named_parameters():
    print(f"Layer: {name} | Size: {param.size()}")

DOACNNCOMPLEX(
  (conv1): Conv2d(2, 256, kernel_size=(3, 3), stride=(2, 2))
  (conv4): Conv2d(256, 256, kernel_size=(2, 2), stride=(1, 1))
  (conv7): Conv2d(256, 256, kernel_size=(2, 2), stride=(1, 1))
  (conv10): Conv2d(256, 256, kernel_size=(2, 2), stride=(1, 1))
  (norm2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (relu6): ReLU()
  (relu9): ReLU()
  (relu12): ReLU()
  (relu15): ReLU()
  (relu18): ReLU()
  (relu21): ReLU()
  (dropout16): Dropout(p=0.2, inplace=False)
  (dropout19): Dropout(p=0.2, inplace=False)
  (dropout22): Dropout(p=0.2, inplace=False)
  (flatten13): Flatten(start_dim=1, end_dim=-1)
  (fc14): Linear(in_features=256, out_features=4096,